# Análise de dados eleições Camaragibe 2024
Esse pequeno projeto tem o objetivo de auxiliar no tratamento de dados na pesquisa eleitoral do município de Camaragibe 2024. O tratamento de dados era feito, diariamente, por meio de aplicações de filtros manuais em diversas planilhas, esse projeto tem objetivo de melhorar essa feitoria, automatizando a filtragem e tratamento desses dados referente as perguntas e respostas adequadas. Observe que o dataset usado é em formato .xlsx (excel), formato fornecido pelo ambiente da pesquisa.

## Base de dados
A base de dados usada é composta por cerca de 24 perguntas, posteriomente processadas, como resultados temos um resultado indicando a quantidade e porcentagem de cada variável do bloco relacionado a pesquisa.

In [432]:
import os
import pandas as pd
df = pd.read_excel("src/CAMARAGIBE - INABI - 06.09.xlsx", sheet_name="DADOS")

data_espontaneas = {}
data_estimuladas = {}

### 1. Sexo
Embora tenha comportamento RU Estimulada, deve-se sempre ter uma resposta válida (feminino/masculino), logo considera os itens vazios como a resposta com mior mostragem.

In [433]:
#1. Sexo
def sexo(coluna="1. SEXO"):
  # Tratamento geral das linhas vazias
  sexo = df[coluna].value_counts()
  valor_mais_frequente = sexo.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_sexo = df[coluna].value_counts()
  porcentagem_sexo = (df[coluna].value_counts(normalize = True) * 100).round(2)
  
  # Exibição em tabela
  resultado = pd.DataFrame({
      'Contagem': contagem_sexo,
      'Porcentagem': porcentagem_sexo.astype(str) + '%'
  })
    
  # Adicionar a linha do total
  total_contagem = contagem_sexo.sum()
  total_porcentagem = porcentagem_sexo.sum()

  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  data_estimuladas[coluna] = resultado

sexo()

### 2. Faixa etaria
Pergunta referente a faixa etária do entrevistado. Faixas etárias:
- 16 a 24 anos
- 25 a 34 anos
- 35 a 44 anos
- 45 a 59 anos
- 60 a 69 anos
- mais de 70 anos

In [434]:
def faixa_etaria(coluna = "2. FAIXA ETÁRIA"):
  # Tratamento geral das linhas vazias
  faixa_etaria = df[coluna].value_counts()
  valor_mais_frequente = faixa_etaria.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_faixa_etaria = df[coluna].value_counts()
  porcentagem_faixa_etaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibir Tabela
  resultado = pd.DataFrame({
      "contagem": contagem_faixa_etaria,
      "porcentagem": porcentagem_faixa_etaria.astype(str) + "%"
  })

  # Adiciona a linha do total
  total_contagem = contagem_faixa_etaria.sum()
  total_porcentagem = porcentagem_faixa_etaria.sum()

  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  data_estimuladas[coluna] = resultado

faixa_etaria()

### 3. Escolaridade
Pergunta referente a escolaridade do entrevistado. Respostas:


In [435]:
def escolaridade(coluna = "3. ESCOLARIDADE"):
  # Tratamento geral das linhas vazias
  escolaridade = df[coluna].value_counts()
  valor_mais_frequente = escolaridade.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_escolaridade = df[coluna].value_counts()
  porcentagem_escolaridade = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibir Tabela
  resultado = pd.DataFrame({
      "contagem": contagem_escolaridade,
      "porcentagem": porcentagem_escolaridade.astype(str) + "%"
  })

  # Adiciona a linha do total
  total_contagem = contagem_escolaridade.sum()
  total_porcentagem = porcentagem_escolaridade.sum()
    
  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  data_estimuladas[coluna] = resultado

escolaridade()

### Tratamento perguntas espontâneas

In [436]:
def tratamento_ru_espontanea(coluna):
  substituicoes = {
        "DIEGO CABRAL": ["diego", "cabral", "diego cabral", "diego alves"],
        "JORGE ALEXANDRE": ["jorge", "jorge alexandre", "alexandre"],
        "FELIPE DANTAS": ["felipe", "dantas", "felipe dantas"],
        "BOSCO": ["bosco"],
        "NENHUM": ["não", "nulo", "NENHUM"],
        "NÃO SABE": ["não sabe", "nao opnou", "nao opinou", "ñ sei"]
    }
  
  # Tratamento geral dos espaços vazios e repetidos
  df[coluna] = df[coluna].fillna("NENHUM")

  # Aplicando a substituição
  for candidato in df[coluna]:
      if (candidato.lower().strip() in substituicoes["DIEGO CABRAL"]):
          df[coluna] = df[coluna].replace(candidato, "DIEGO CABRAL")

      elif (candidato.lower().strip() in substituicoes["JORGE ALEXANDRE"]):
          df[coluna] = df[coluna].replace(candidato, "JORGE ALEXANDRE")

      elif (candidato.lower().strip() in substituicoes["FELIPE DANTAS"]):
          df[coluna] = df[coluna].replace(candidato, "FELIPE DANTAS")

      elif (candidato.lower().strip() in substituicoes["BOSCO"]):
          df[coluna] = df[coluna].replace(candidato, "BOSCO")

      elif (candidato.lower().strip() in substituicoes["NENHUM"]):
          df[coluna] = df[coluna].replace(candidato, "NENHUM")

      elif (candidato.lower().strip() in substituicoes["NÃO SABE"]):
          df[coluna] = df[coluna].replace(candidato, "NÃO SABE/NÃO RESPONDEU")
        
  contagem_votaria = df[coluna].value_counts()
  porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibição da tabela
  resultado = pd.DataFrame({
      "contagem": contagem_votaria,
      "porcentagem": porcentagem_votaria.astype(str) + "%"
  })

  # Total
  total_contagem = contagem_votaria.sum()
  total_porcentagem = porcentagem_votaria.sum()
    
  if (total_porcentagem != 100.00):
      ajuste = 100.00 - total_porcentagem
      total_porcentagem += ajuste

  resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]

  data_estimuladas[coluna] = resultado

perguntas = [
    "5. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSE HOJE, EM QUAL CANDIDATO VOCÊ VOTARIA? (RU ESPONTÂNEA)",
    "17. A PREFEITA NADEGI NÃO PODERÁ SER MAIS CANDIDATA NA PRÓXIMA ELEIÇÃO. QUEM É O CANDIDATO A PREFEITO QUE ELA IRÁ APOIAR? (RU ESPONTÂNEA)"
]

for coluna in perguntas:
    tratamento_ru_espontanea(coluna)

Exception ignored in: <function ZipFile.__del__ at 0x7f1aa90e3ba0>
Traceback (most recent call last):
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x7f1aa90e3ba0>
Traceback (most recent call last):
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x7f1aa90e3ba0>
Traceback (most recent call last):
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/home/bosco/anaconda3/lib/python3.12/zipfile/__init__.py", line 

### Tratamento de dados das perguntas RU Estimulada

In [437]:
def tratamento_ru_estimulada(coluna):
  # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem = df[coluna].value_counts()
    porcentagem = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem,
        "porcentagem": porcentagem.astype(str) + "%"
    })

    # Total
    total_contagem = contagem.sum()
    total_porcentagem = porcentagem.sum()
    
    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste

    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]

    data_espontaneas[coluna] = resultado

perguntas = [
    "4. COMO VOCÊ CLASSIFICARIA O SEU INTERESSE NAS ELEIÇÕES MUNICIPAIS DESTE ANO? (RU ESTIMULADA)",
    "6. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUAIS DESSES CANDIDATOS VOCÊ VOTARIA? (RU ESTIMULADA)",
    "7. INDEPENDENTE DA SUA INTENÇÃO DE VOTOS, QUEM VOCÊ ACREDITA QUE ESTEJA MAIS PREPARADO PARA SER PREFEITO DE CAMARAGIBE? (RU ESTIMULADA)",
    "8. INDEPENDENTE DA SUA INTENÇÃO DE VOTOS, QUEM VOCÊ ACREDITA QUE GANHARÁ A ELEIÇÃO PARA PREFEITO DE CAMARAGIBE? (RU ESTIMULADA)",
    "9. EM RELAÇÃO A SUA INTENÇÃO DE VOTO PARA PREFEITO DE CAMARAGIBE, HOJE VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "10. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE DIEGO CABRAL, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "11. (EXCETO PARA QUEM NÃO CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE DIEGO CABRAL: (RU ESTIMULADA)",
    "12. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE JORGE ALEXANDRE, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "13. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE JORGE ALEXANDRE: (RU ESTIMULADA)",
    "14. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE BOSCO, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "15. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE BOSCO: (RU ESTIMULADA)",
    "16. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUEM VOCÊ VOTARIA? (RU ESTIMULADA)",
    "18. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, VOCÊ VOTARIA EM DIEGO CABRAL SE O PRESIDENTE LULA APOIAR? (RU ESTIMULADA)",
    "19. DE MANEIRA GERAL, COMO VOCÊ AVALIA A GESTÃO DA GOVERNADORA RAQUEL LYRA? (RU ESTIMULADA)",
    "20. SOBRE A GESTÃO DA PREFEITA NADEGI, COMO O (A) SR. (A) AVALIA A SUA ADMINISTRAÇÃO A FRENTE DA PREFEITURA DE CAMARAGIBE? (RU ESTIMULADA)",
    "21. DE MANEIRA GERAL, SOBRE A FORMA DE ADMINISTRAR DA PREFEITA NADEGI, O (A) SR. (A) APROVA OU DESAPROVA? (RU ESTIMULADA)",
    "22. EM QUAL DESSES CANDIDATOS VOCÊ NÃO VOTARIA DE MANEIRA ALGUMA? (RU ESTIMULADA)",
    "23. (NÃO PEA) DAS ALTERNATIVAS QUE VOU LER, QUAL MELHOR REPRESENTA A SUA SITUAÇÃO EM RELAÇÃO AO TRABALHO?",
    "24. RENDA (PESSOAL)",
    "25. RELIGIÃO"
    ]

for coluna in perguntas:
    tratamento_ru_estimulada(coluna)

In [438]:
def relacao_intencao (coluna = "9. EM RELAÇÃO A SUA INTENÇÃO DE VOTO PARA PREFEITO DE CAMARAGIBE, HOJE VOCÊ DIRIA QUE: (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

relacao_intencao()

,contagem,porcentagem
"9. EM RELAÇÃO A SUA INTENÇÃO DE VOTO PARA PREFEITO DE CAMARAGIBE, HOJE VOCÊ DIRIA QUE: (RU ESTIMULADA)",,
"É DEFINITIVA, NÃO MUDAREI DE OPINIÃO",47,83.93%
NÃO SABE/NÃO RESPONDEU,4,7.14%
"NÃO É DEFINITIVA, MUDAREI DE OPINIÃO",4,7.14%
TALVEZ MUDAREI DE OPINIÃO,1,1.79%
Total,56,100.00%


In [439]:
def sobre_candidatura_diego_cabral(coluna = "10. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE DIEGO CABRAL, VOCÊ DIRIA QUE: (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

sobre_candidatura_diego_cabral()

,contagem,porcentagem
"10. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE DIEGO CABRAL, VOCÊ DIRIA QUE: (RU ESTIMULADA)",,
VOTARIA COM CERTEZA,22,39.29%
NÃO SABE/NÃO RESPONDEU,15,26.79%
TALVEZ VOTARIA,7,12.5%
NÃO O CONHECE O SUFICIENTE PARA OPINAR,6,10.71%
NÃO VOTARIA DE JEITO NENHUM,6,10.71%
Total,56,100.00%


In [440]:
def imagem_possitiva_negativa_diego_cabral(coluna = "11. (EXCETO PARA QUEM NÃO CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE DIEGO CABRAL: (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

imagem_possitiva_negativa_diego_cabral()

,contagem,porcentagem
11. (EXCETO PARA QUEM NÃO CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE DIEGO CABRAL: (RU ESTIMULADA),,
POSITIVA,24,42.86%
NEM POSITIVA NEM NEGATIVA,18,32.14%
NÃO SABE/NÃO RESPONDEU,12,21.43%
NEGATIVA,2,3.57%
Total,56,100.00%


In [441]:
def sobre_jorge_alexandre(coluna = "12. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE JORGE ALEXANDRE, VOCÊ DIRIA QUE: (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

sobre_jorge_alexandre()

,contagem,porcentagem
"12. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE JORGE ALEXANDRE, VOCÊ DIRIA QUE: (RU ESTIMULADA)",,
NÃO SABE/NÃO RESPONDEU,16,28.57%
NÃO VOTARIA DE JEITO NENHUM,14,25.0%
VOTARIA COM CERTEZA,12,21.43%
TALVEZ VOTARIA,8,14.29%
NÃO O CONHECE O SUFICIENTE PARA OPINAR,6,10.71%
Total,56,100.00%


In [442]:
def imagem_positiva_negativa_bosco(coluna = "15. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE BOSCO: (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

imagem_positiva_negativa_bosco()

,contagem,porcentagem
15. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE BOSCO: (RU ESTIMULADA),,
NEM POSITIVA NEM NEGATIVA,22,39.29%
NÃO SABE/NÃO RESPONDEU,20,35.71%
NEGATIVA,13,23.21%
POSITIVA,1,1.79%
Total,56,100.00%


In [443]:
def eleicoes_prefeito_hoje_quem_votaria(coluna = "16. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUEM VOCÊ VOTARIA? (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

eleicoes_prefeito_hoje_quem_votaria()

,contagem,porcentagem
"16. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUEM VOCÊ VOTARIA? (RU ESTIMULADA)",,
"DIEGO CABRAL, APOIADO PELO PREFEITO DE RECIFE JOÃO CAMPOS",23,41.07%
"JORGE ALEXANDRE, APOIADO POR MARCELO GOUVEIA",13,23.21%
NÃO SABE/NÃO RESPONDEU,10,17.86%
"FELIPE DANTAS, APOIADO POR ANDERSON FERREIRA",5,8.93%
NENHUM DELES,4,7.14%
"BOSCO, APOIADO POR DUDU DA FONTE",1,1.79%
Total,56,100.00%


In [444]:
def avaliacao_gestao_governadora_raquel_lyra(coluna = "19. DE MANEIRA GERAL, COMO VOCÊ AVALIA A GESTÃO DA GOVERNADORA RAQUEL LYRA? (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

avaliacao_gestao_governadora_raquel_lyra()

,contagem,porcentagem
"19. DE MANEIRA GERAL, COMO VOCÊ AVALIA A GESTÃO DA GOVERNADORA RAQUEL LYRA? (RU ESTIMULADA)",,
DESAPROVO,23,41.07%
APROVO,22,39.29%
NÃO SABE/NÃO RESPONDEU,11,19.64%
Total,56,100.00%


In [445]:
def avaliacao_prefeitura_nadegi(coluna = "20. SOBRE A GESTÃO DA PREFEITA NADEGI, COMO O (A) SR. (A) AVALIA A SUA ADMINISTRAÇÃO A FRENTE DA PREFEITURA DE CAMARAGIBE? (RU ESTIMULADA)"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

avaliacao_prefeitura_nadegi()

,contagem,porcentagem
"20. SOBRE A GESTÃO DA PREFEITA NADEGI, COMO O (A) SR. (A) AVALIA A SUA ADMINISTRAÇÃO A FRENTE DA PREFEITURA DE CAMARAGIBE? (RU ESTIMULADA)",,
RUIM,16,28.57%
PÉSSIMA,12,21.43%
BOA,11,19.64%
NÃO SABE/NÃO RESPONDEU,8,14.29%
REGULAR,6,10.71%
ÓTIMA,3,5.36%
Total,56,100.00%


In [446]:
def religiao(coluna = "25. RELIGIÃO"):
    # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem_votaria = df[coluna].value_counts()
    porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem_votaria,
        "porcentagem": porcentagem_votaria.astype(str) + "%"
    })

    # Total
    total_contagem = contagem_votaria.sum()
    total_porcentagem = porcentagem_votaria.sum()

    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste


    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]
    
    return resultado

religiao()

,contagem,porcentagem
25. RELIGIÃO,,
CATÓLICA,27,48.21%
EVANGÉLICA,19,33.93%
ESPÍRITA,7,12.5%
SEM RELIGIÃO,2,3.57%
NÃO SABE/NÃO RESPONDEU,1,1.79%
Total,56,100.00%


# Exportando análise

In [447]:
data_final = {**data_espontaneas, **data_estimuladas}

with pd.ExcelWriter('results/resultados_tratamento.xlsx') as writer:
    for titulo, dataset in data_final.items():
        # Limitar o nome da aba a 30 caracteres, se necessário
        nome_aba = titulo[:30]
        dataset.to_excel(writer, sheet_name=nome_aba, index=False)